In [12]:
import numpy
import serial
from serial.serialutil import SerialException

In [23]:
PORT = "/dev/ttyUSB2"
BAUDRATE = 115200
TIMEOUT = 1  # seconds

ser = serial.Serial(
    port=PORT,
    baudrate=BAUDRATE,
    timeout=TIMEOUT
)

if ser.is_open:
    print(f"Connected successfully to {PORT} at {BAUDRATE} baud")

Connected successfully to /dev/ttyUSB2 at 115200 baud


In [ ]:
#HEX: 9C A0 9C A0 9C 9F 9A 9F 9A 99 9E 9D 9E 9D 9B 9B 9E 9B 9E 9F 9E 9D 9E 9D 9F

[108, 108, 111, 32, 87, 111, 114, 108, 100, 10, 13, 83, 117, 99, 99, 101, 115, 115, 102, 117]
